In [1]:
import numpy as np
import tensorflow as tf
import gzip
import pickle
import sys
from ddm.run_split import SplitMnistGenerator
from ddm.alg.cla_models_multihead import MFVI_IBP_NN, Vanilla_NN
#from ddm.alg.vcl import run_vcl, run_vcl_ibp
from copy import deepcopy

import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt

W0809 20:04:02.053158 140684216964928 deprecation_wrapper.py:119] From /home/skessler/Projects/IBP_BNN/ddm/alg/cla_models_multihead.py:9: The name tf.set_random_seed is deprecated. Please use tf.compat.v1.set_random_seed instead.



# IBP network

In [2]:
hidden_size = [50]
batch_size = 512
no_epochs = 100
alpha0 = 50.0
tau0=1.0 # initial temperature
ANNEAL_RATE=0.000
MIN_TEMP=0.5

# Run vanilla VCL
tf.set_random_seed(12)
np.random.seed(1)

coreset_size = 0
data_gen = SplitMnistGenerator()
single_head=False
in_dim, out_dim = data_gen.get_dims()
x_testsets, y_testsets = [], []
for task_id in range(data_gen.max_iter):
    x_train, y_train, x_test, y_test = data_gen.next_task()
    x_testsets.append(x_test)
    y_testsets.append(y_test)

    # Set the readout head to train
    head = 0 if single_head else task_id
    bsize = x_train.shape[0] if (batch_size is None) else batch_size
    
    # Train network with maximum likelihood to initialize first model
    if task_id == 0:
        ml_model = Vanilla_NN(in_dim, hidden_size, out_dim, x_train.shape[0])
        ml_model.train(x_train, y_train, task_id, no_epochs, bsize)
        mf_weights = ml_model.get_weights()
        mf_variances = None
        mf_betas = None
        ml_model.close_session()

    # Train on non-coreset data
    mf_model = MFVI_IBP_NN(in_dim, hidden_size, out_dim, x_train.shape[0], prev_means=mf_weights, 
                           prev_log_variances=mf_variances, prev_betas=mf_betas,alpha0=alpha0,
                           learning_rate=0.01, temp_prior=1.0, no_pred_samples=100)
    mf_model.train(x_train, y_train, head, no_epochs, bsize, tau0=tau0, 
                   anneal_rate=ANNEAL_RATE, min_temp=MIN_TEMP)
    mf_weights, mf_variances, mf_betas = mf_model.get_weights()

    # Incorporate coreset data and make prediction
#     x_coresets = []
#     y_coresets = []
#     acc = get_scores(mf_model, x_testsets, y_testsets, x_coresets, y_coresets, hidden_size, no_epochs, single_head, batch_size)
#     all_acc = concatenate_results(acc, all_acc)

    mf_model.close_session()

W0809 20:04:02.795602 140684216964928 deprecation_wrapper.py:119] From /home/skessler/Projects/IBP_BNN/ddm/alg/cla_models_multihead.py:52: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0809 20:04:02.798237 140684216964928 deprecation_wrapper.py:119] From /home/skessler/Projects/IBP_BNN/ddm/alg/cla_models_multihead.py:163: The name tf.truncated_normal is deprecated. Please use tf.random.truncated_normal instead.

W0809 20:04:02.856984 140684216964928 deprecation_wrapper.py:119] From /home/skessler/Projects/IBP_BNN/ddm/alg/cla_models_multihead.py:57: The name tf.train.AdamOptimizer is deprecated. Please use tf.compat.v1.train.AdamOptimizer instead.

W0809 20:04:02.968654 140684216964928 deprecation_wrapper.py:119] From /home/skessler/Projects/IBP_BNN/ddm/alg/cla_models_multihead.py:61: The name tf.global_variables_initializer is deprecated. Please use tf.compat.v1.global_variables_initializer instead.

W0809 20:04:02.970085 140684216964928 deprecat

Epoch: 0001 cost= 0.216955772
Epoch: 0006 cost= 0.004734276
Epoch: 0011 cost= 0.002391301
Epoch: 0016 cost= 0.001469344
Epoch: 0021 cost= 0.000950934
Epoch: 0026 cost= 0.000599450
Epoch: 0031 cost= 0.000395737
Epoch: 0036 cost= 0.000270286
Epoch: 0041 cost= 0.000203017
Epoch: 0046 cost= 0.000151774
Epoch: 0051 cost= 0.000122883
Epoch: 0056 cost= 0.000095334
Epoch: 0061 cost= 0.000077950
Epoch: 0066 cost= 0.000065157
Epoch: 0071 cost= 0.000054726
Epoch: 0076 cost= 0.000046292
Epoch: 0081 cost= 0.000041603
Epoch: 0086 cost= 0.000034075
Epoch: 0091 cost= 0.000029798
Epoch: 0096 cost= 0.000026382
Optimization Finished!


W0809 20:04:08.081196 140684216964928 deprecation.py:323] From /home/skessler/Projects/IBP_BNN/ddm/alg/cla_models_multihead.py:511: to_int32 (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.cast` instead.


beta a: (50,)
beta b: (50,)
logpis: (1, ?, 50)
act: (100, ?, 50, 1)
pre: <unknown>
beta a: (50,)
beta b: (50,)
logpis: (1, 784, 50)
beta a: ()
beta b: ()
logpis: (1, 784, 50)


W0809 20:04:08.491629 140684216964928 deprecation.py:323] From /home/skessler/Projects/IBP_BNN/ddm/alg/cla_models_multihead.py:493: div (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


beta a: (2,)
beta b: (2,)
logpis: (1, 50, 2)
beta a: ()
beta b: ()
logpis: (1, 50, 2)
beta a: (50,)
beta b: (50,)
logpis: (1, ?, 50)
act: (10, ?, 50, 1)
pre: <unknown>
beta a: (50,)
beta b: (50,)
logpis: (1, ?, 50)
act: (100, ?, 50, 1)
pre: <unknown>


W0809 20:04:09.004714 140684216964928 deprecation.py:323] From /home/skessler/anaconda2/envs/py36/lib/python3.6/site-packages/tensorflow/python/ops/math_grad.py:1205: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0809 20:04:10.804988 140684216964928 deprecation_wrapper.py:119] From /home/skessler/Projects/IBP_BNN/ddm/alg/cla_models_multihead.py:501: The name tf.train.Saver is deprecated. Please use tf.compat.v1.train.Saver instead.

W0809 20:04:10.843743 140684216964928 deprecation_wrapper.py:119] From /home/skessler/Projects/IBP_BNN/ddm/alg/cla_models_multihead.py:581: The name tf.summary.scalar is deprecated. Please use tf.compat.v1.summary.scalar instead.

W0809 20:04:10.892964 140684216964928 deprecation_wrapper.py:119] From /home/skessler/Projects/IBP_BNN/ddm/alg/cla_models_multihead.py:595: The name tf

nu: <unknown>
mag: <unknown>
shape of z: <unknown>
1
(512, 784)
(512, 2)


InvalidArgumentError: Shapes of all inputs must match: values[0].shape = [] != values[1].shape = [50]
	 [[node summaries/strided_slice_3/stack (defined at /home/skessler/Projects/IBP_BNN/ddm/alg/cla_models_multihead.py:571) ]]

Original stack trace for 'summaries/strided_slice_3/stack':
  File "/home/skessler/anaconda2/envs/py36/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/skessler/anaconda2/envs/py36/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/skessler/anaconda2/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/skessler/anaconda2/envs/py36/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/skessler/anaconda2/envs/py36/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 497, in start
    self.io_loop.start()
  File "/home/skessler/anaconda